<a href="https://colab.research.google.com/github/nbilasals/music_recommendation/blob/main/music_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music Recommendation (Spotify)

## Import Required Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from google.colab import files
import os
import shutil
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.express as px
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')

## Data Preparation

### Loading Data

In [3]:
# uploaded = files.upload()

In [4]:
# kaggle_dir = os.path.expanduser('~/.kaggle')
# os.makedirs(kaggle_dir, exist_ok=True)

# # Pindahkan file kaggle.json
# shutil.move('kaggle.json', os.path.join(kaggle_dir, 'kaggle.json'))

# # Atur permission file
# os.chmod(os.path.join(kaggle_dir, 'kaggle.json'), 0o600)

In [5]:
# !kaggle datasets download -d maharshipandya/-spotify-tracks-dataset

In [6]:
# !unzip /content/-spotify-tracks-dataset.zip

In [46]:
#Load Data
data = pd.read_csv("dataset.csv")

In [47]:
data

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,...,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
113996,113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,...,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
113997,113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,...,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music
113998,113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,...,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music


In [48]:
#Print genres in dataset
unique_genres = data['track_genre'].unique()
print(unique_genres)
print(f"Number of unique genres: {len(unique_genres)}")

['acoustic' 'afrobeat' 'alt-rock' 'alternative' 'ambient' 'anime'
 'black-metal' 'bluegrass' 'blues' 'brazil' 'breakbeat' 'british'
 'cantopop' 'chicago-house' 'children' 'chill' 'classical' 'club' 'comedy'
 'country' 'dance' 'dancehall' 'death-metal' 'deep-house' 'detroit-techno'
 'disco' 'disney' 'drum-and-bass' 'dub' 'dubstep' 'edm' 'electro'
 'electronic' 'emo' 'folk' 'forro' 'french' 'funk' 'garage' 'german'
 'gospel' 'goth' 'grindcore' 'groove' 'grunge' 'guitar' 'happy'
 'hard-rock' 'hardcore' 'hardstyle' 'heavy-metal' 'hip-hop' 'honky-tonk'
 'house' 'idm' 'indian' 'indie-pop' 'indie' 'industrial' 'iranian'
 'j-dance' 'j-idol' 'j-pop' 'j-rock' 'jazz' 'k-pop' 'kids' 'latin'
 'latino' 'malay' 'mandopop' 'metal' 'metalcore' 'minimal-techno' 'mpb'
 'new-age' 'opera' 'pagode' 'party' 'piano' 'pop-film' 'pop' 'power-pop'
 'progressive-house' 'psych-rock' 'punk-rock' 'punk' 'r-n-b' 'reggae'
 'reggaeton' 'rock-n-roll' 'rock' 'rockabilly' 'romance' 'sad' 'salsa'
 'samba' 'sertanejo' 'show

In [50]:
print("Numbers of Rows and Columns:")
data.shape

Numbers of Rows and Columns:


(114000, 21)

In [49]:
data.columns

Index(['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name',
       'popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'track_genre'],
      dtype='object')

### Dropping Duplicate Values

In [51]:
# Check for duplicates based on 'track_id'
duplicates = data[data.duplicated(subset=['track_id'])]

# Display the duplicated rows
print("Duplicated rows based on 'track_id':")
print(duplicates)

# Count the number of duplicates
num_duplicates = len(duplicates)
print(f"\nNumber of duplicated track IDs: {num_duplicates}")

Duplicated rows based on 'track_id':
        Unnamed: 0                track_id  \
1925          1925  0CDucx9lKxuCZplLXUz0iX   
2155          2155  2aibwv5hGXSgw7Yru8IYTO   
3000          3000  5E30LdtzQTGqRvNd7l6kG5   
3002          3002  2K7xn816oNHJZ0aVqdQsha   
3003          3003  2QjOHCTQ1Jl3zawyYOpxh6   
...            ...                     ...   
113572      113572  1saXUvvFlAQaefZUFVmhCn   
113605      113605  1Q5jFp1g2Ns4gBsHRpcqhu   
113617      113617  71dLJx3qHOTQMTvvoE2dmd   
113619      113619  6OG5TBCmuTOuWCzSGsETrE   
113641      113641  7xsirhcgFWOnItsGuBfrv9   

                                           artists  \
1925                        Buena Onda Reggae Club   
2155                         Red Hot Chili Peppers   
3000                             The Neighbourhood   
3002                             The Neighbourhood   
3003                             The Neighbourhood   
...                                            ...   
113572                   Bethel 

In [57]:
# Display rows before dropping duplicates
print("Rows before dropping duplicates:")
print(data.shape[0])

Rows before dropping duplicates:
114000


In [54]:
# Drop duplicate rows based on all columns
data_no_duplicates = data.drop_duplicates(subset='track_id')

After this process, the dataset now has fewer rows as duplicates have been removed. This refined data will be used for further steps.

In [58]:
# Display rows after dropping duplicates
print("Rows after dropping duplicates:")
data_no_duplicates.shape[0]

Rows after dropping duplicates:


89741

### Dropping Missing Values

In [59]:
# Display number of missing values in each column
print("Missing values per column:")
print(data_no_duplicates.isnull().sum())

Missing values per column:
Unnamed: 0          0
track_id            0
artists             1
album_name          1
track_name          1
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64


In [60]:
# Drop rows with any missing values
data = data_no_duplicates.dropna()

In this step, rows containing missing values are removed. Since the number of missing values is very small compared to the overall dataset, the deletion method is deemed sufficient and will not significantly affect data quality.

In [61]:
# Display the shape of the DataFrame after dropping missing values
print("Shape of DataFrame after dropping missing values:")
data.shape

Shape of DataFrame after dropping missing values:


(89740, 21)

### Drop Unrelevant Columns

In [15]:
# Drop the 'Unnamed: 0' column
data = data.drop('Unnamed: 0', axis=1)

# Now 'data' DataFrame does not have the 'Unnamed: 0' column
data.columns

Index(['track_id', 'artists', 'album_name', 'track_name', 'popularity',
       'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature', 'track_genre'],
      dtype='object')

After preprocessing, the dataset has been cleaned and optimized for modeling:

- Initial Data: 114,000 rows, 21 columns.
- Duplicates Removed: 24,259 rows dropped, leaving 89,741 rows.
- Missing Values: 1 row with missing artists, album_name, or track_name removed.
- Final Shape: 89,740 rows, 20 columns (dropped irrelevant Unnamed: 0).
- Unique Genres: 114, ensuring diverse recommendations.

The dataset is now ready for building a robust music recommendation system.

## Modelling

### 1. Using K-Nearest Neighbors based on Song Features

These features should represent song characteristics.

In [67]:
# Select the relevant features for the model
X = data[['danceability', 'energy', 'loudness', 'tempo', 'valence']]

Create and Train the KNN Model

In [68]:
from sklearn.neighbors import NearestNeighbors

# Initialize the KNN model
knn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')

# Fit the model on the features
knn.fit(X)

NearestNeighbors(algorithm='ball_tree', n_neighbors=10)

Define the Recommendation Function

In [71]:
def recommend_tracks_by_name(track_name, n_recommendations=5):
    """
    Recommend similar tracks based on KNN using track_name.

    Parameters:
        track_name (str): Name of the track for which recommendations are made.
        n_recommendations (int): Number of recommendations to return (default 5).

    Returns:
        pd.DataFrame: Recommended tracks with track name, artist, and genre.
    """
    # Check if track_name exists in the dataset
    if track_name in data['track_name'].values:
        idx_data = data[data['track_name'] == track_name].index[0]
        idx_X = X.index.get_loc(idx_data)
        distances, indices = knn.kneighbors([X.iloc[idx_X]], n_neighbors=n_recommendations + 1)
        recommended_tracks = data.iloc[indices[0][1:]].copy()
        recommended_tracks['distance'] = distances[0][1:]  # Menyimpan jarak
        return recommended_tracks[['track_name', 'artists', 'track_genre', 'danceability', 'energy', 'loudness', 'tempo', 'valence', 'distance']]
    else:
        print(f"Track name '{track_name}' not found in the dataset.")
        return None


 Test the Recommendation Function

In [74]:
data[data['track_name'] == "Call It Fate, Call It Karma"][['track_name', 'artists', 'track_genre', 'danceability', 'energy', 'loudness', 'tempo', 'valence']]

,track_name,artists,track_genre,danceability,energy,loudness,tempo,valence
2463,"Call It Fate, Call It Karma",The Strokes,alt-rock,0.544,0.241,-14.779,109.437,0.359


In [79]:
# Test the function with a specific track ID
name_track='Call It Fate, Call It Karma'
recommendations =recommend_tracks_by_name(name_track)

print("Top-5 Recommendation from:", name_track)
# Display the recommendations
recommendations

Top-5 Recommendation from: Call It Fate, Call It Karma


,track_name,artists,track_genre,danceability,energy,loudness,tempo,valence,distance
93125,Студенточка,Pyotr Leshchenko,romance,0.487,0.222,-14.956,109.195,0.477,0.327761
52637,Same Ole Me,George Jones,honky-tonk,0.759,0.223,-15.000,109.595,0.415,0.351411
52555,Last Date,Floyd Cramer,honky-tonk,0.742,0.311,-14.760,109.679,0.608,0.406239
93178,"Нет, не любил он",Valentina Ponomaryova,romance,0.456,0.131,-14.903,109.109,0.189,0.414372
108458,Mi Bandoneon Y Yo,Rubén Juárez;Carlos Garcia,tango,0.573,0.249,-14.947,109.519,0.778,0.459798


By using Euclidean distance, we can effectively measure how "close" two tracks are based on their audio characteristics, and sort them to find the most similar tracks to the input one.

### 2. Using Cosine Similarity based on Track Genre

In [20]:
# Inisialisasi TfidfVectorizer
tf = TfidfVectorizer()

# Melakukan perhitungan idf pada data
tf.fit(data['track_genre'])

# Mapping array dari fitur index integer ke fitur nama
tf.get_feature_names_out()

array(['acoustic', 'afrobeat', 'age', 'alt', 'alternative', 'ambient',
       'and', 'anime', 'bass', 'black', 'bluegrass', 'blues', 'brazil',
       'breakbeat', 'british', 'cantopop', 'chicago', 'children', 'chill',
       'classical', 'club', 'comedy', 'country', 'dance', 'dancehall',
       'death', 'deep', 'detroit', 'disco', 'disney', 'drum', 'dub',
       'dubstep', 'edm', 'electro', 'electronic', 'emo', 'film', 'folk',
       'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth',
       'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard',
       'hardcore', 'hardstyle', 'heavy', 'hip', 'honky', 'hop', 'house',
       'idm', 'idol', 'indian', 'indie', 'industrial', 'iranian', 'jazz',
       'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal',
       'metalcore', 'minimal', 'mpb', 'music', 'new', 'opera', 'pagode',
       'party', 'piano', 'pop', 'power', 'progressive', 'psych', 'punk',
       'reggae', 'reggaeton', 'rock', 'rockabilly', 'roll', 'romance',
  

In [63]:
# Melakukan fit lalu ditransformasikan ke bentuk matrix
tfidf_matrix = tf.fit_transform(data['track_genre'])

# Melihat ukuran matrix tfidf
tfidf_matrix.shape

(89740, 114)

In [64]:
# Mengubah vektor tf-idf dalam bentuk matriks dengan fungsi todense()
tfidf_matrix.todense()

matrix([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.70710678],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.70710678],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.70710678]])

In [65]:
#Tampilkan dalam DataFrame
pd.DataFrame(
    tfidf_matrix.todense(),
    columns=tf.get_feature_names_out(),
    index=data.track_name
).sample(22, axis=1).sample(10, axis=0)

,dubstep,party,and,turkish,dancehall,pagode,indian,pop,classical,roll,...,happy,swedish,singer,alternative,soul,bass,german,rockabilly,folk,mandopop
track_name,,,,,,,,,,,,,,,,,,,,,
Have a Good Time,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Starseed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Uno,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alles anders,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I'm This,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Assobio Sombrio - Remix,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ainda sei te amar,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bedtime Baby,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
It's Gonna Go Away,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Sekarang, kita akan menghitung derajat kesamaan (similarity degree) antar musik dengan teknik cosine similarity.

In [66]:
# Menghitung cosine similarity pada matrix tf-idf
cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [25]:
# Membuat dataframe dari variabel cosine_sim dengan baris dan kolom berupa nama
cosine_sim_df = pd.DataFrame(cosine_sim, index=data['track_name'], columns=data['track_name'])
print('Shape:', cosine_sim_df.shape)

# Melihat similarity matrix pada setiap resto
cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

Shape: (89740, 89740)


track_name,Shatter - Edit,Stirred Illusion,Days of the Week,I Am Mine,Fool's Paradise - Joris Voorn Remix
track_name,,,,,
Change Us,0.0,0.0,0.0,0.0,0.000000
Čtyři Slunce,0.0,0.0,0.0,0.0,0.000000
opening pattern,0.0,0.0,0.0,0.0,0.000000
Papo Furado,0.0,0.0,0.0,0.0,0.000000
Mercy,0.0,0.0,0.0,0.0,0.000000
Stranger in the House (with Elvis Costello),0.0,0.0,0.0,0.0,0.000000
Feel Surreal Ends the Feel Surreal,0.0,0.0,0.0,0.0,0.645461
Desliga e Vem / Gamei / Gandaia (feat. Péricles & Chrigor) - Ao Vivo,0.0,0.0,0.0,0.0,0.000000
Primal Energy (Haunted Grounds),0.0,0.0,0.0,0.0,0.000000


In [40]:
def music_recommendations(nama_track, similarity_data=cosine_sim_df, items=data[['track_name','artists', 'track_genre']], k=5):
    # Mengambil data dengan menggunakan argpartition untuk melakukan partisi secara tidak langsung sepanjang sumbu yang diberikan
    # Dataframe diubah menjadi numpy
    # Range(start, stop, step)
    index = similarity_data.loc[:,nama_track].to_numpy().argpartition(
        range(-1, -k, -1))

    # Mengambil data dengan similarity terbesar dari index yang ada
    # Convert 'index' to 1D array before using it to index 'similarity_data.columns'
    closest = similarity_data.columns[index[-1:-(k+2):-1].ravel()]

    # Drop nama_resto agar nama resto yang dicari tidak muncul dalam daftar rekomendasi
    closest = closest.drop(nama_track, errors='ignore')

    return pd.DataFrame(closest).merge(items).head(k)

In [33]:
data[data.track_name.eq('Call It Fate, Call It Karma')]

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
2463,3Y4rUyw7XBCK6hGHCOt6rp,The Strokes,Comedown Machine,"Call It Fate, Call It Karma",72,204773,False,0.544,0.241,4,-14.779,0,0.03,0.983,0.774,0.101,0.359,109.437,4,alt-rock


In [45]:
# Test the function with a specific track ID
name_track='Call It Fate, Call It Karma'
recommendations =music_recommendations(name_track)

print("Top-5 Recommendation from:", name_track)
# Display the recommendations
recommendations

Top-5 Recommendation from: Call It Fate, Call It Karma


,track_name,artists,track_genre
0,Razón - En Vivo,Los Caligaris,alt-rock
1,Razón - En Vivo,Los Caligaris,alt-rock
2,Baby Came Home 2 / Valentines,The Neighbourhood,alt-rock
3,Love It If We Made It,The 1975,alt-rock
4,Butterfly,Crazy Town,alt-rock
